# 데이터 분할

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

chunksize = 5*(10**6)
#13.55gb

for cnt, chunk in enumerate(pd.read_csv('/content/drive/MyDrive/zip/12051164-493d-4dcc-8cd1-36e2b4a9ab3a.zip', iterator=True, chunksize=chunksize)):
    if cnt == 10: #5~5599999
        df_120_5 = pd.DataFrame()
        df_120_5 = chunk
    elif cnt < 10:
        continue
    else:
        break

In [ ]:
df_120_5

inDate  ...                                     tableAndColumn
50000000  2021-12-31 15:39:47.423  ...             [{tablename=MainData, columns=[Data]}]
50000001  2021-12-31 15:39:47.423  ...                                                NaN
50000002  2021-12-31 15:39:47.423  ...                [{tablename=Growth, columns=[Exp]}]
50000003  2021-12-31 15:39:47.424  ...                                                NaN
50000004  2021-12-31 15:39:47.426  ...             [{tablename=MainData, columns=[Data]}]
...                           ...  ...                                                ...
54999995  2021-12-31 17:01:27.298  ...          [{tablename=UserInfo, columns=[LastMap]}]
54999996  2021-12-31 17:01:27.298  ...     [{tablename=DailyMission, columns=[Mission0]}]
54999997  2021-12-31 17:01:27.299  ...                                                NaN
54999998  2021-12-31 17:01:27.300  ...                                                NaN
54999999  2021-12-31 17:01:27.301  ...  [{tablename=user_info, columns=[best_stage, go...

[5000000 rows x 6 columns]

In [ ]:
import re

# 열 삭제
df_120_5 = df_120_5.drop("tableAndColumn",axis=1)

#method 중 PUT,GET만 사용
df_120_5 = df_120_5[(df_120_5['method']=='PUT')|(df_120_5['method']=='GET')]

#url에서 특성 빼내기
df_120_5['category'] = df_120_5['url'].apply(lambda x : x.split('/')[2])

#gameInfo ==> gameinfo
li = []
for i in df_120_5['category']:
    i = re.sub('gameInfo','gameinfo', i)
    li.append(i)

df_120_5['category'] = li

#inDate 특성 시간까지만 확인하기
df_120_5['time'] = df_120_5['inDate'].apply(lambda x : x[10:13])

#inDate에서 날짜 값만 지정
df_120_5['inDate']=df_120_5['inDate'].apply(lambda x : x[:11])

# method 열 삭제
df_120_5 = df_120_5.drop("method",axis=1)

# url => action
df_120_5 = df_120_5.rename(columns={"url":"action"})

df_120_5 = df_120_5.groupby(by=['gamer_id', 'game_id', 'inDate', 'category', 'time']).count()

df_120_5

action
gamer_id                                           game_id inDate      category     time        
0001b282486938b2735e00ea3a65a6af52d648c15bfaa89... 380     2021-12-31  gameinfo      15        2
000872f22a55a15f6ced715d91dfe7d286d0e28e540559a... 1916    2021-12-31  chart         16        1
                                                                       gameinfo      16       14
                                                                       gamer         16        1
                                                                       notice        16        1
...                                                                                          ...
fffa81067d1c4f36c302b3f8457379c3df8beaf9b3f021f... 1585    2021-12-31  social        15       16
                                                                                     16       56
fffacd2059e511c368b4b927a34d1893c464ea13dd30b15... 1585    2021-12-31  isAliveToken  15        2
                                                                                     16        5
                                                                                     17        1

[107538 rows x 1 columns]

In [ ]:
df_120_5.to_csv('df_120_5_54999.csv', mode='w')